### Finding coordinates of from Replan 1 and Replan 2

###### In this project:
- First of all, we read two databases. 
- Then, merge both of them with indicator to see which common rows between two databses. 
- Then, individual "currentStopName" finds.
- The coordinations (latitude and longitude) of these individual "currentStopName"s are found. 
- Then it will be checked to see whether all the coordinations have been found in Switzerland.
- Afterwards, the coordidations of each city will be merged with the master file which is "indirectEffect" dataframe. 
- Finally, the number of repitition time count for each individual "currentStopName. 

###### To run the code, please follow the following steps:
- Click on "Kernel" in the menu bar
- Select "Restart & Run All" 
- Hold on untill a number assigns to each cell. 

In [41]:
# library
import pandas as pd
import numpy as np
#import googlemaps
from geopy.geocoders import Nominatim

In [42]:
# the name of the app (no need to change)
geolocator = Nominatim(user_agent="defultName")

# STEP 1 read the files

In [43]:
df1 = pd.read_csv("replan_current_information.txt",sep="\t", error_bad_lines=False, index_col =None)
df2 = pd.read_csv("replan_replan_when_boardingDenied.txt",sep="\t", error_bad_lines=False, index_col =None)

In [44]:
# read the replan.nodisruption.txt (Read the second databae)
#df2 = pd.read_csv("replan_replan_when_boardingDenied.txt",sep="\t", error_bad_lines=False, index_col =None)
len(df2)
df2.head()


,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit
0,0,1394529300,57600.0,16:0:0,T-2,"Zürich, Einfangstrasse",VBZ_line62,15661_080,NFB_VBZ_140751,"Zürich, Zehntenhausplatz",1,1
1,0,1152035700,57600.0,16:0:0,T-2,"Zürich, Klusplatz",VBZ_line703,23724_001,NFB_VBZ_142004,"Zürich, Carl-Spitteler-Strasse",1,1
2,0,1016616800,57600.0,16:0:0,T-2,"Mellingen Heitersberg, Bahnhof",PAG_line362,36202_038,BUS_PAG_95481,"Mellingen, Meierhofer",1,1
3,0,1613689101,57600.0,16:0:0,T-2,"Zürich, Lerchenhalde",VBZ_line80,06342_167,NFB_VBZ_138485,"Zürich, Schumacherweg",1,1
4,0,919327800,57606.0,16:0:6,T-2,"Zürich, Klusplatz",VBZ_line33,00881_001,NFB_VBZ_137778,"Zürich, Hölderlinsteig",1,1


In [45]:
print(df1.shape)
print(df2.shape)

(1880, 12)
(1496, 12)


# STEP 2 just keep T2 in file one

In [46]:
df1 = df1[df1["triggerType"]=="T-2"]

In [47]:
print(df1.shape)

(1749, 12)


In [48]:
len(df1.personId.unique())

625

In [49]:
len(df2.personId.unique())

554

# STEP 3 Remove duplicates in each file

In [50]:
df1 = df1.drop_duplicates(subset=['personId', 'triggerType',
       'currentStopName', 'transitLine', 'transitRoute', 'vehicle',
       'nextStopName', 'passengersSize', 'passengerCapacit'] , keep='first')
df1.shape

(1006, 12)

In [51]:
len(df1.personId.unique())

625

In [52]:
df2 = df2.drop_duplicates(subset=['personId', 'triggerType',
       'currentStopName', 'transitLine', 'transitRoute', 'vehicle',
       'nextStopName', 'passengersSize', 'passengerCapacit'] , keep='first')
df2.shape

(842, 12)

In [53]:
len(df2.personId.unique())

554

# Step 4 Compare the two files
1. All values are the same in all columns

In [54]:
# merged the two database with indicators demoestrating which row belongs to which database
df3 = pd.merge(df1,df2, how='outer', indicator=True)
len(df3)
df3.head()


,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit,_merge
0,0,1016616800,57600.0,16:0:0,T-2,"Mellingen Heitersberg, Bahnhof",PAG_line362,36202_038,BUS_PAG_95481,"Mellingen, Meierhofer",1,1,both
1,0,1152035700,57600.0,16:0:0,T-2,"Zürich, Klusplatz",VBZ_line703,23724_001,NFB_VBZ_142004,"Zürich, Carl-Spitteler-Strasse",1,1,both
2,0,1613689101,57600.0,16:0:0,T-2,"Zürich, Lerchenhalde",VBZ_line80,06342_167,NFB_VBZ_138485,"Zürich, Schumacherweg",1,1,both
3,0,1394529300,57600.0,16:0:0,T-2,"Zürich, Einfangstrasse",VBZ_line62,15661_080,NFB_VBZ_140751,"Zürich, Zehntenhausplatz",1,1,both
4,0,919327800,57606.0,16:0:6,T-2,"Zürich, Klusplatz",VBZ_line33,00881_001,NFB_VBZ_137778,"Zürich, Hölderlinsteig",1,1,both


# ALL the Same

In [55]:
# Merge all the values between the two data frames that are exactly the same (with inner)
df_all_same = pd.merge(df1, df2 , on= ['iteration', 'personId', 'time', 'OutTime', 'triggerType',
       'currentStopName', 'transitLine', 'transitRoute', 'vehicle',
       'nextStopName', 'passengersSize', 'passengerCapacit'])

In [56]:
len(df_all_same)

790

In [57]:
len(df_all_same.personId.unique())

525

# Just in the Current file

In [58]:
# Filter the merged databse into two database, indirecedEffect and Tocheck databases.
df_indirectEffect = df3[df3["_merge"]=="left_only"]
df_indirectEffect.to_excel("indirectEffect.xlsx")
df_indirectEffect.head()

,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit,_merge
15,0,1557445300,57852.0,16:4:12,T-2,Zürich Oerlikon,SBB_S6_8503105-8503504,18622_003,S_SBB_04321,Zürich Seebach,0,30,left_only
16,0,1192519601,57900.0,16:5:0,T-2,"Zürich, Bahnhofquai/HB",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Stampfenbachplatz",1,1,left_only
17,0,914693600,57906.0,16:5:6,T-2,"Zürich, Stampfenbachplatz",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Nordstrasse",1,1,left_only
44,0,1488941500,58440.0,16:14:0,T-2,"Zürich, Rebbergsteig",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Kempfhofsteig",1,1,left_only
54,0,1248398200,58667.0,16:17:47,T-2,Zürich Oerlikon,SBB_S24_8502204-8506000,20420_001,S_SBB_05050,Zürich Flughafen,0,30,left_only


In [59]:
len(df_indirectEffect)

216

In [60]:
len(df_indirectEffect.personId.unique())

130

# Just in the repaln file

In [21]:
# Filter the merged databse into two database, indirecedEffect and Tocheck databases.
df_to_check = df3[df3["_merge"]=="right_only"]
df_to_check.to_excel("to_check.xlsx")
df_to_check.head()

,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit,_merge
1006,0,1491268300,59894.0,16:38:14,T-2,"Schwerzenbach ZH, Dorf",VBZ_line704,30235_067,NFB_VBZ_142999,"Fällanden, Industrie",1,1,right_only
1007,0,1374678600,60180.0,16:43:0,T-2,"Zürich, Albisriederplatz",VBZ_line72,04444_001,NFB_VBZ_138727,"Zürich, Friedhof Sihlfeld",1,1,right_only
1008,0,1374678600,60492.0,16:48:12,T-2,"Zürich, Albisriederplatz",VBZ_line72,04444_001,NFB_VBZ_138726,"Zürich, Friedhof Sihlfeld",1,1,right_only
1009,0,866277600,60840.0,16:54:0,T-2,"Zürich, Escher-Wyss-Platz",VBZ_line33,06358_153,NFB_VBZ_140790,"Zürich, Rosengartenstrasse",1,1,right_only
1010,0,1374678600,60972.0,16:56:12,T-2,"Zürich, Albisriederplatz",VBZ_line72,04444_001,NFB_VBZ_138725,"Zürich, Friedhof Sihlfeld",1,1,right_only


In [22]:
len(df_to_check)

52

In [23]:
len(df_to_check.personId.unique())

38

# Step 5

In [24]:
df_indirectEffect_no_duplication = df_indirectEffect.drop_duplicates(keep="first").reset_index(drop=True)
print ("the length of the city_df is {}".format(len(df_indirectEffect_no_duplication)))
df_indirectEffect_no_duplication.head()

the length of the city_df is 216


,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit,_merge
0,0,1557445300,57852.0,16:4:12,T-2,Zürich Oerlikon,SBB_S6_8503105-8503504,18622_003,S_SBB_04321,Zürich Seebach,0,30,left_only
1,0,1192519601,57900.0,16:5:0,T-2,"Zürich, Bahnhofquai/HB",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Stampfenbachplatz",1,1,left_only
2,0,914693600,57906.0,16:5:6,T-2,"Zürich, Stampfenbachplatz",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Nordstrasse",1,1,left_only
3,0,1488941500,58440.0,16:14:0,T-2,"Zürich, Rebbergsteig",VBZ_line46,07822_178,NFB_VBZ_139413,"Zürich, Kempfhofsteig",1,1,left_only
4,0,1248398200,58667.0,16:17:47,T-2,Zürich Oerlikon,SBB_S24_8502204-8506000,20420_001,S_SBB_05050,Zürich Flughafen,0,30,left_only


In [25]:
# find the indivisual "currentStopName" and remove the repitition
city_list = list(set(df_indirectEffect_no_duplication["currentStopName"].to_list()))
city_df = pd.DataFrame(city_list, columns =['currentStopName'])
print ("the length of the city_df is {}".format(len(city_df)))
city_df.head()

the length of the city_df is 66


,currentStopName
0,"Zürich, Im Ebnet"
1,"Adlikon b. R., Pächterried"
2,"Lufingen, Unterdorf"
3,"Glattbrugg, Frohbühlstrasse"
4,"Bassersdorf, Bahnhof"


In [26]:
%%time
# expand the city_df dataframe by finding the coordination of each "currentStopName"
city_df["latitude"] = None
city_df["longitude"] = None

for idx in range(len(city_df)):
    try:
        address = city_df["currentStopName"][idx] 
        location = geolocator.geocode(address, addressdetails=True)
        city_df.at[idx, "latitude"] = location.raw["lat"]
        city_df.at[idx, "longitude"] = location.raw["lon"]
    except:
        address = city_df["currentStopName"][idx].split(", ")[0] + ", switzerland"
        try:
            location = geolocator.geocode(address, addressdetails=True)
            city_df.at[idx, "latitude"] = location.raw["lat"]
            city_df.at[idx, "longitude"] = location.raw["lon"]
        except:
            address = city_df["currentStopName"][idx].split(", ")[0] + ", ch"
            try:
                location = geolocator.geocode(address, addressdetails=True)
                city_df.at[idx, "latitude"] = location.raw["lat"]
                city_df.at[idx, "longitude"] = location.raw["lon"]
            except:
                pass         
    finally:
        print ("% " + str(idx/len(city_df) * 100))
print ("the length of the city_df is {}".format(len(city_df)))
city_df.head() 

% 0.0
% 1.5151515151515151
% 3.0303030303030303
% 4.545454545454546
% 6.0606060606060606
% 7.575757575757576
% 9.090909090909092
% 10.606060606060606
% 12.121212121212121
% 13.636363636363635
% 15.151515151515152
% 16.666666666666664
% 18.181818181818183
% 19.696969696969695
% 21.21212121212121
% 22.727272727272727
% 24.242424242424242
% 25.757575757575758
% 27.27272727272727
% 28.78787878787879
% 30.303030303030305
% 31.818181818181817
% 33.33333333333333
% 34.84848484848485
% 36.36363636363637
% 37.878787878787875
% 39.39393939393939
% 40.909090909090914
% 42.42424242424242
% 43.93939393939394
% 45.45454545454545
% 46.96969696969697
% 48.484848484848484
% 50.0
% 51.515151515151516
% 53.03030303030303
% 54.54545454545454
% 56.060606060606055
% 57.57575757575758
% 59.09090909090909
% 60.60606060606061
% 62.121212121212125
% 63.63636363636363
% 65.15151515151516
% 66.66666666666666
% 68.18181818181817
% 69.6969696969697
% 71.21212121212122
% 72.72727272727273
% 74.24242424242425
% 75.75

,currentStopName,latitude,longitude
0,"Zürich, Im Ebnet",47.2878684,8.5576186
1,"Adlikon b. R., Pächterried",None,None
2,"Lufingen, Unterdorf",None,None
3,"Glattbrugg, Frohbühlstrasse",None,None
4,"Bassersdorf, Bahnhof",None,None


In [27]:
# change the data type of "latitude" and "longitude" columns from string to float
city_df["latitude"] = city_df["latitude"].astype(float)
city_df["longitude"] = city_df["longitude"].astype(float)

In [28]:
# Show the rows where "latitude" and "longitude" could not be found.  
city_df[city_df["latitude"].isnull()].head()

,currentStopName,latitude,longitude
1,"Adlikon b. R., Pächterried",NaN,NaN
2,"Lufingen, Unterdorf",NaN,NaN
3,"Glattbrugg, Frohbühlstrasse",NaN,NaN
4,"Bassersdorf, Bahnhof",NaN,NaN
6,"Wetzikon ZH, Bahnhof",NaN,NaN


In [29]:
# Switzerland boundaries
lat_min = 45.827413
lat_max = 47.775553
lon_min = 5.952471
lon_max = 10.467563

In [30]:
# list of the stations found out of the country
out_of_country = city_df[(city_df["latitude"]>=lat_max) | (city_df["latitude"]<=lat_min) | \
                            (city_df["longitude"]>=lon_max)| (city_df["longitude"]<=lon_min)]
# take out the index
out_of_country_index = out_of_country.index.to_list()
print ("list of indices need to be corrected:" , out_of_country_index)
out_of_country.head()

list of indices need to be corrected: []


,currentStopName,latitude,longitude


In [31]:
# here we apply the loop once again over the places where found out of the country of Switzerland 
for idx in out_of_country_index:
    city_df.loc[idx, "latitude"] = np.nan
    city_df.loc[idx, "longitude"] = np.nan
    try:
        new_address = ", ".join(city_df.loc[idx, "currentStopName"].split(", ")[::-1]) + ", swiss" 
        location = geolocator.geocode(new_address, addressdetails=True)
        city_df.at[idx, "latitude"] = location.raw["lat"]
        city_df.at[idx, "longitude"] = location.raw["lon"]
    except:
        pass

In [32]:
# change the data type of "latitude" and "longitude" columns from string to float
city_df["latitude"] = city_df["latitude"].astype(float)
city_df["longitude"] = city_df["longitude"].astype(float)

In [33]:
# Let's see if there are still a place not found by the code
city_df[city_df["latitude"].isnull()].head()

,currentStopName,latitude,longitude
1,"Adlikon b. R., Pächterried",NaN,NaN
2,"Lufingen, Unterdorf",NaN,NaN
3,"Glattbrugg, Frohbühlstrasse",NaN,NaN
4,"Bassersdorf, Bahnhof",NaN,NaN
6,"Wetzikon ZH, Bahnhof",NaN,NaN


In [34]:
# here the we merge the city_df and df_indirectEffect
merged_df = pd.merge(left=df_indirectEffect_no_duplication, right=city_df)
merged_df.head()

,iteration,personId,time,OutTime,triggerType,currentStopName,transitLine,transitRoute,vehicle,nextStopName,passengersSize,passengerCapacit,_merge,latitude,longitude
0,0,1557445300,57852.0,16:4:12,T-2,Zürich Oerlikon,SBB_S6_8503105-8503504,18622_003,S_SBB_04321,Zürich Seebach,0,30,left_only,47.412036,8.544317
1,0,1248398200,58667.0,16:17:47,T-2,Zürich Oerlikon,SBB_S24_8502204-8506000,20420_001,S_SBB_05050,Zürich Flughafen,0,30,left_only,47.412036,8.544317
2,0,1060282700,60012.0,16:40:12,T-2,Zürich Oerlikon,SBB_S9_8503125-8503424,18920_002,S_SBB_04557,Glattbrugg,0,30,left_only,47.412036,8.544317
3,0,1086246801,60012.0,16:40:12,T-2,Zürich Oerlikon,SBB_S9_8503125-8503424,18920_002,S_SBB_04557,Glattbrugg,0,30,left_only,47.412036,8.544317
4,0,914693600,60192.0,16:43:12,T-2,Zürich Oerlikon,SBB_S21_8503000-8503526,20162_008,S_SBB_04998,Zürich Affoltern,0,30,left_only,47.412036,8.544317


In [35]:
# save the "indirectEffect" into a excel file
merged_df.to_excel("indirectEffect_visual.xlsx")

############### Data Analysis #######################

In [36]:
merged_df["category"] = "Red"
merged_df1 = merged_df[["currentStopName", "category",  "latitude","longitude","personId", "triggerType"]]
merged_df1.head()

,currentStopName,category,latitude,longitude,personId,triggerType
0,Zürich Oerlikon,Red,47.412036,8.544317,1557445300,T-2
1,Zürich Oerlikon,Red,47.412036,8.544317,1248398200,T-2
2,Zürich Oerlikon,Red,47.412036,8.544317,1060282700,T-2
3,Zürich Oerlikon,Red,47.412036,8.544317,1086246801,T-2
4,Zürich Oerlikon,Red,47.412036,8.544317,914693600,T-2


In [37]:
personId_df = merged_df1.groupby(["personId", "currentStopName","latitude","longitude", "triggerType"]).count()
personId_df.head(10)

category
personId  currentStopName              latitude  longitude triggerType          
759147200 Brüttisellen, Ob. Wangenstr. 47.424258 8.630773  T-2                 1
760196600 Zürich Oerlikon              47.412036 8.544317  T-2                 1
775420400 Zürich HB                    47.378101 8.539364  T-2                 1
777639500 Zürich HB                    47.378101 8.539364  T-2                 1
797580500 Opfikon, Austrasse           47.432267 8.577998  T-2                 1
          Zürich Hardbrücke, Bahnhof   47.385161 8.517393  T-2                 1
807239400 Niederglatt ZH, Nöschikon    47.495077 8.492097  T-2                 1
811642600 Zürich Oerlikon              47.412036 8.544317  T-2                 1
822365800 Zürich Oerlikon              47.412036 8.544317  T-2                 1
828805500 Opfikon, Austrasse           47.432267 8.577998  T-2                 2

In [38]:
personId_df.to_excel("indreictEffect_based_number_of_personId.xlsx")

In [39]:
## based on the stops
StopName_df = merged_df1.groupby(["currentStopName","latitude","longitude", "triggerType", "category"]).count()
StopName_df.head(50)

,,,,,personId
currentStopName,latitude,longitude,triggerType,category,
"Bachenbülach, alte Post",47.505065,8.546070,T-2,Red,3
"Bergdietikon, Dünni",47.389840,8.386526,T-2,Red,1
"Brüttisellen, Ob. Wangenstr.",47.424258,8.630773,T-2,Red,1
"Glattbrugg, Bahnhof",47.432631,8.564789,T-2,Red,1
"Glattbrugg, Lättenwiesen",47.432631,8.564789,T-2,Red,1
"Höri, Schulhaus",47.508503,8.506701,T-2,Red,1
"Küsnacht ZH, Bühlstrasse",47.319365,8.610749,T-2,Red,1
"Niederglatt ZH, Nöschikon",47.495077,8.492097,T-2,Red,1
"Niederhasli, Hofstetterstrasse",47.481607,8.492832,T-2,Red,1


In [40]:
StopName_df.to_excel("only_T2_indirectEffect_basedon_Station.xlsx")

### The End